In [15]:
import numpy as np
import sys

In [9]:
!head X_public.csv

Id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10
1,4.77397060,6.28870916,3.64784479,5.13400888,9.52229691,9.16195107,6.35711718,7.17296934,1.41602564,6.06968880
2,5.12932396,8.39112186,6.12639856,2.96031618,6.37552261,5.24287176,4.93583012,9.72775078,2.92516804,7.71357727
3,5.25995350,0.86055845,1.92213845,6.63226891,8.90232658,3.48892927,0.64171320,0.20023048,4.57701731,0.63095838
4,5.31606436,0.39280343,4.37637615,9.31835079,9.30809784,7.20952320,2.84293413,7.38534307,6.39978838,3.54048657
5,9.56468201,5.88640165,6.57304049,8.58676338,4.39559889,9.23969841,3.98436666,8.14766884,6.84218502,9.10972023
6,7.86002111,3.07457852,4.47033596,2.26106620,1.87533104,2.76234674,5.56443739,4.16501284,1.69607091,9.06803894
7,2.94160366,2.32261539,5.84488487,2.44412732,1.52389789,7.32148504,1.25474894,7.93470383,1.64101923,7.45071411
8,6.39458370,7.59734869,0.93480474,1.34902418,5.20210075,0.78232139,0.69906396,2.04655075,4.61420488,8.19677258
9,8.705398

In [10]:
my_data = np.genfromtxt('X_public.csv', delimiter=',', skip_header=1, usecols=(1,2,3,4,5,6,7,8,9,10))

In [29]:
initial_x = my_data[2000:3000]

In [13]:
def get_str_oracle (X):
    return "{} {} {} {} {} {} {} {} {} {}".format (X[0], X[1], X[2], X[3], X[4], X[5], X[6], X[7], X[8], X[9])

In [ ]:
y = []
for i, x in enumerate (initial_x):
    args = get_str_oracle (x)
    res = !./Oracle.static $args
    sys.stderr.write ("\ri:{} res:{}".format (i, res[0]))
    try:
        y.append (float (res[0]))
    except ValueError as vr:
        print vr, "on res ",res[0], "data", args
        y.append (None)

i:689 res:27986256146659.09375000000113984.000000000000000000000000

In [ ]:
y = np.array (y)

In [ ]:
np.savetxt ("y_2000_2999.csv", y)